# Toy Radix Lagrangean decomposition validation.
# Guaranteed primal recovery via branch and bound.

- Obviously, full space solution still gets a better incumbent if solved to small enough gap.
- LB for LP relaxation is the same for decomposition, obviously.
- Decomposition gets better integer LB sooner
- Incorporate branching strategies and heuristics from full space approach to recover better feasible primal.

In [1]:
#OBJECTIVE    = 'minsse'
OBJECTIVE    = 'minerr'
REG_WEIGHT   = 0. #1e-4
MAX_NONZERO  = None #48*2
PREVENT_ZERO = True
POWERS       = [-1,0,1]
TWO_PHASE    = True
SIMPLE_MM    = True

if SIMPLE_MM:
    BASE_MODEL_FILE = '/home/laurence/ME/models/e_coli_core_mm_simple.json'
else:
    BASE_MODEL_FILE = '/home/laurence/ME/models/e_coli_core_mm.json'

In [2]:
BASE_MODEL_FILE = '/home/laurence/ME/models/e_coli_core_pc.json'

In [3]:
%load_ext line_profiler

In [4]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
#pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from cobra.io import load_json_model
from six import iteritems
import numpy as np
import cobra

In [5]:
ijomc = load_json_model(BASE_MODEL_FILE)

In [6]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [7]:
df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if ijomc.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

#N_CONDS = len(conds)
#N_CONDS = ['succinate','malate']
#N_CONDS = ['glucose','acetate']
N_CONDS = ['glucose','acetate','fructose']

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])

if hasattr(N_CONDS,'__iter__'):
    df_conds = df_conds[ df_conds.cond.isin(N_CONDS)]
    N_CONDS = len(N_CONDS)
else:
    if N_CONDS==1:
        df_conds = df_conds[ df_conds.cond=='acetate']
    elif N_CONDS<=3:
        df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'][0:N_CONDS])]
    else:
        df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

df_conds.loc[ (df_conds.cond=='acetate') & (df_conds.rxn=='EX_ac_e'), 'lb'] = -20


In [8]:
def sol_to_kdict(sol_final):
    var_cons_dict = estk.var_cons_dict
    powers = estk.powers
    digits = estk.digits
    radix  = estk.radix

    kfit_dict = {}
    for group_id, var_dict in iteritems(var_cons_dict):
        var = var_dict[0]
        cons = var_dict[1]
        a0  = var_dict[0][2]
        kfit = 0.
        for l,pwr in enumerate(powers):
            for k,digit in enumerate(digits):            
                yid = 'binary_%s%s%s' % (group_id,k,l)
                y   = sol_final[yid]
                kfit += y*a0*radix**pwr*digit
        kfit_dict[group_id] = kfit

    kfit_changed = [(k,v, abs(v-a0)/a0) for k,v in iteritems(kfit_dict) if abs(v-a0)/a0>1e-6]
    print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
    
    return kfit_dict

def compute_perf(kfit_dict, ijofit):
    #----------------------------------------
    # Starting from basal model
    perrs = []
    errs_fit = []
    errs_unfit = []
    csrcs = df_conds.cond.unique()
    rows = []
    for csrc in csrcs:        
        crowding = ijofit.metabolites.crowding
        #--------------------------------------------------
        # Constrain to medium
        df_condi = df_conds[ df_conds.cond==csrc]    
        for i,row in df_condi.iterrows():
            rid = row['rxn']
            rxn = ijofit.reactions.get_by_id(rid)
            rxn.lower_bound = row['lb']
            rxn.upper_bound = row['ub']
        # Get unfit growth rate first    
        ijofit.optimize(solver='gurobi')
        mu_unfiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x

        #--------------------------------------------------
        # Fitted
        for rid,kfit in iteritems(kfit_dict):
            rxn = ijofit.reactions.get_by_id(rid)
            rxn.add_metabolites({crowding:kfit}, combine=False)

        ijofit.optimize(solver='gurobi')

        mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
        mu_fiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x

        err0= 100*(mu_unfiti-mu_measi)/mu_measi
        err = 100*(mu_fiti - mu_measi)/mu_measi
        derr= 100*(abs(err)-abs(err0))/abs(err0)
        perrs.append(err)
        errs_unfit.append(mu_unfiti - mu_measi)
        errs_fit.append(mu_fiti - mu_measi)
        print('Cond=%s. mu_meas=%g. mu_sim=%g (unfit=%g, error=%.3g%%). Error=%.3g%% (%.3g%% change)' % (
            csrc, mu_measi, mu_fiti, mu_unfiti, err0, err, derr))
        for i,row in df_condi.iterrows():
            rid = row['rxn']
            rxn = ijofit.reactions.get_by_id(rid)        
            print('\t%s uptake=%g' % (rxn.id, rxn.x))

        rows.append({'substrate':csrc, 'mu_fit':mu_fiti, 'mu_unfit':mu_unfiti, 'mu_meas':mu_measi})

    perrs = np.array(perrs)
    errs_fit = np.array(errs_fit)
    errs_unfit = np.array(errs_unfit)
    tot_err = sum(abs(errs_fit))
    tot_err0 = sum(abs(errs_unfit))
    mape = np.mean(abs(perrs))
    mdape = np.median(abs(perrs))
    print("Absolute total error: %g"%(tot_err))
    print("Total error change: %g%%"%( 100*(tot_err-tot_err0)/tot_err0  ))
    print("Absolute percent error: %g%% -- %g%%"%(min(abs(perrs)), max(abs(perrs))))
    print("Mean abs percent error = %g%%"%(mape))
    print("Median abs percent error = %g%%"%(mdape))
    
    f_change = (tot_err-tot_err0)/tot_err0
    perf_dict = {'tot_err':tot_err, 'err_change':f_change, 'mape':mape, 'mdape':mdape}    
    df_results = pd.DataFrame(rows)
    
    return perf_dict, df_results

# Make radix problem

In [9]:
from dynamicme import decomposition


In [10]:
from dynamicme.estimate import RadixEstimator

In [11]:
df_Y = df_meas.rename(columns={'growth_rate_1_h':'output', 'substrate':'cond'})
df_Y.loc[:,'output_id'] = 'BIOMASS_Ecoli_core_w_GAM'
df_X = df_conds

# Each Lagrange submodel should start with the optimum

In [12]:
from dynamicme.decomposition.LagrangeMaster import LagrangeMaster
from dynamicme.decomposition.LagrangeSubmodel import LagrangeSubmodel

In [13]:
conds = df_conds.cond.unique()
sub_dict = {}
for cond in conds:
    df_Xk = df_X[ df_X.cond==cond]
    df_Yk = df_Y[ df_Y.cond==cond]
    estk = RadixEstimator()
    if SIMPLE_MM:
        cons_var_pairs = 'crowding'
    else:
        cons_var_pairs = [(cons, rxn) for cons in ijomc.metabolites.query('^enz_cap') for rxn in cons.reactions if rxn.id==cons.id.replace('enz_cap','e')]
    estk.fit(ijomc, df_Xk, df_Yk, objective=OBJECTIVE, 
             fit_constraint_id = cons_var_pairs,
             reg_weight = REG_WEIGHT, max_nonzero_binaries=MAX_NONZERO, optimize=False, powers=POWERS)
    sub = LagrangeSubmodel(estk.stacker.model, cond, Q=estk.stacker.Q)    
    sub_dict[cond] = sub

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1


In [14]:
master = LagrangeMaster(estk.stacker.model)
master.add_submodels(sub_dict)
master.covered_dict = estk.covered_dict

In [15]:
for sub in sub_dict.values():
    sub.model.Params.MIPGapAbs = 1e-3
    sub.model.Params.MIPGap    = 1e-3

# B&B to recover primal

In [ ]:
from dynamicme.decomposition.LagrangeBB import LagrangeBB

In [ ]:
master.time_limit = 2*3600
#master.max_iter = 500
master.verbosity = 1
master.print_iter = 10
# master.gaptol = 0.02
master.gaptol = 1e-3
master.absgaptol = 1e-3
master.delta_min  = 1e-10
master.delta_mult = 0.5
#master.bundle_mult = 0.1
#master.model.Params.NumericFocus = 3   # the QP eventually runs into numerical issues

master.scale_factor = 1.

heuristics = ['reduced_cost','parsimonious','best_rounding']

lagrangeBB = LagrangeBB(master, heuristics)

lagrangeBB.branch_rule = 'dispersion_random'

sol_master = lagrangeBB.optimize(two_phase=TWO_PHASE, only_relaxed=False, feasible_methods=[])
df_phase2 = pd.DataFrame(master.log_rows)

0         . NODES TO EXPLORE: 1
    Iter                    UB                    LB       gap relgap(%)   penalty                       Time(s)
  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Dual   Feasible        Sub       Best                                   total    master       sub
       0       1000     1e+100          0          0      1000       100         0  0.061177  0.002942  0.046632
      10      0.186     1e+100     -49.75          0     0.186       100    0.0698  0.439724  0.001897  0.035686
      20    0.06591     1e+100     -795.9          0   0.06591       100   0.00893  0.781172  0.004508  0.030276
      30    0.04819     1e+100      -1953          0   0.04819       100  5.17e-05  1.153269  0.008895  0.033084
      40    0.04809     1e+100     -18.35          0   0.04809       100  4.88e-10  1.567076  0.012520  0.029889
      50    0.04809     1e+100     -116.9          0   0.04809  

/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:526 UserWarning: Solution available but Master solver status=suboptimal (13).


  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Dual   Feasible        Sub       Best                                   total    master       sub
       0  6.746e-05     1e+100   -0.07294   -0.07294     0.073 1.082e+05  0.000118  2.001798  0.177862  1.804527
       4 -4.653e-06     1e+100   0.000258   0.000258         0         0  3.37e-09  6.379705  0.314560  0.072830
0 NODES DROPPED BY PRUNING: z > 0.192654
y[111] = 1.0
BRANCHING ON y[111]: binary_CS00 with dispersion(cost)=0.22222222222222224 in [0.0, 0.22222222222222224]
Child0: [0.0, 0.0], Child1: [1.0, 1.0]
1         . NODES TO EXPLORE: 2
    Iter                    UB                    LB       gap relgap(%)   penalty                       Time(s)
  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Dual   Feasible        Sub       Best                                   total

/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: 1.73610487764e-07 < 0.00233535254508



       0  1.736e-07     1e+100   0.002335   0.002335  0.002335 1.344e+06  8.36e-13  0.344452  0.139851  0.166815
Master status=2
Subobjs=[0.0009889035041367063, 0.0007721010998843477, 0.0005758294358460147]
total penalty=4.598866691920491e-13
UB<LB: 1.00500074041e-07 < 0.00233683403987
      10 -2.668e-05     1e+100  0.0003189   0.002337  0.002364      8858  9.37e-07  7.417240  0.625276  0.021536
      20 -0.0001235     1e+100  0.0003939   0.002337   0.00246      1992  2.33e-05  11.41135  0.324743  0.021957
      30 -0.0004901     1e+100    0.00121   0.002337  0.002827     576.8  6.73e-07  15.73387  0.447674  0.020444
      40  -0.005492     1e+100   -0.06712   0.004299  0.009791     178.3  2.26e-07  19.32232  0.302089  0.243762
      50 -0.0005121     1e+100   0.004144   0.004299  0.004811     939.5  3.37e-09  23.05687  0.593310  0.040383
Master status=13
Subobjs=[-3.86348464443482, -3.863195771967412, 7.7278764157711635]
total penalty=7.515573210397119e-13

...


    3670 -7.814e-05     1e+100   0.003774    0.01306   0.01314 1.682e+04  1.72e-13  1402.065  0.294009  0.020697
Master status=13
Subobjs=[-8.139310897750818, -8.14209903809811, 16.285645645943333]
total penalty=6.840238350554475e-14
UB<LB: -0.000543367492583 < 0.00423571009441
Master status=13
Subobjs=[-8.117351448414851, -8.120139643265944, 16.24172699952709]
total penalty=7.559499163133337e-15
UB<LB: -0.000543490554833 < 0.00423590784629
Master status=13
Subobjs=[-7.676804502309821, -7.679539149598654, 15.361863803379833]
total penalty=7.256684884296099e-13
UB<LB: -0.0032588949731 < 0.00552015147136
Master status=13
Subobjs=[-8.136046262818297, -8.138938769469206, 16.27875950784923]
total penalty=5.302055432749194e-13
UB<LB: -7.82981815064e-05 < 0.00377447556173
    3680 -0.0005432     1e+100   0.004236    0.01306   0.01361      2504  3.57e-12  1406.335  0.502078  0.020967
Master status=13
Subobjs=[-8.122033179523001, -8.124821328449556, 16.251090392447168]
total penalty=5.09852079

/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -7.80445228656e-05 < 0.00377413691616
/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -0.000543321770238 < 0.0042357088835
/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -0.000543466502108 < 0.00423587866509
/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -0.000543522612702 < 0.0042357788351
/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -7.82943142212e-05 < 0.00377479511098
/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -7.80704330055e-05 < 0.00377448319179
/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -0.0

UB<LB: -0.000543603276242 < 0.00423598869554
Master status=13
Subobjs=[-8.135199573925107, -8.138092143218998, 16.277066149026638]
total penalty=4.901500513477797e-13
UB<LB: -7.81883543368e-05 < 0.00377443188253
    3900 -0.0005437     1e+100   0.004236    0.01306   0.01361      2503  1.13e-12  1504.022  0.340738  0.020667
Master status=13
Subobjs=[-8.119608471762376, -8.122396747409795, 16.246240349287916]
total penalty=7.258169260998491e-13
UB<LB: -0.000542839823538 < 0.00423513011575
Master status=13
Subobjs=[-8.200684776596063, -8.20357719472615, 16.40803595262298]
total penalty=2.466464672124338e-13
UB<LB: -7.83590669471e-05 < 0.00377398130077
Master status=13
Subobjs=[-8.141505296816717, -8.144293611704825, 16.29003467009028]
total penalty=1.9193929533828188e-13
UB<LB: -0.000543228705283 < 0.00423576156874
Master status=13
Subobjs=[-8.121529630868537, -8.124317883600781, 16.25008342276257]
total penalty=6.703577482555812e-14
UB<LB: -0.000543510850548 < 0.00423590829325
    3910  

/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -0.00325867592949 < 0.0055215112848
/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -7.9318934695e-05 < 0.00377477751984
/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -0.000543567309933 < 0.00423588851456
/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -0.00325883944572 < 0.00552168738372
/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -0.000544998970554 < 0.00423760098046
/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -0.000543410322166 < 0.00423605344524
/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition/LagrangeMaster.py:586 UserWarning: UB<LB: -0.000

Subobjs=[-8.119847059485252, -8.12263535698655, 16.24671850719659]
total penalty=2.903316916405993e-13
UB<LB: -0.000543636788166 < 0.00423609072479
Master status=13
Subobjs=[-8.192978053813093, -8.195870904676553, 16.39262334572379]
total penalty=3.26625628210896e-13
UB<LB: -7.83844407124e-05 < 0.00377438723415
Master status=13
Subobjs=[-8.144445205971294, -8.147233320725604, 16.29591431138357]
total penalty=5.185703834820358e-13
UB<LB: -0.000543335601265 < 0.00423578468667
Master status=13
Subobjs=[-8.125685640146912, -8.128473820412239, 16.258395422468595]
total penalty=2.046168346359102e-13
UB<LB: -0.000543424512671 < 0.00423596190944
Master status=13
Subobjs=[-8.117495161762122, -8.120283340050946, 16.242014855668828]
total penalty=8.281454418147778e-14
UB<LB: -0.000543695448982 < 0.00423635385576
    4130 -0.0005437     1e+100   0.004236    0.01306   0.01361      2502  8.28e-14  1608.134  0.342666  0.020776
Master status=2
Subobjs=[-8.018749053743647, -8.021669985408948, 16.044007

## Feasible?

In [ ]:
lagrangeBB.tree.nodes

In [ ]:
node = lagrangeBB.tree.nodes[5]

In [ ]:
node.verbosity=1
node.reset_problem()
node.inherit_changes()

In [ ]:
node.parent.bound_dict

In [ ]:
node.bound_dict

In [ ]:
node.parent.verbosity=1
node.parent.inherit_changes()

## Check best feasible solution

In [ ]:
if sol_master:
    sol_final = sol_master.copy()

In [ ]:
mdl0 = load_json_model(BASE_MODEL_FILE)
kd = sol_to_kdict(sol_master)
perf, df_results = compute_perf(kd, mdl0)

In [ ]:
from scipy import stats

dsplot = df_results.melt(['substrate','mu_meas'])
dsplot.loc[dsplot.variable=='mu_fit','model'] = 'Fit'
dsplot.loc[dsplot.variable=='mu_unfit','model'] = 'Unfit'
df_perf = dsplot.groupby('variable').apply(lambda x: pd.Series({
    'mdape':100*np.median(abs(x['value']-x['mu_meas'])/abs(x['mu_meas'])),
    'sse':sum( (x['value']-x['mu_meas'])**2 ),
    'rho':stats.spearmanr(x['mu_meas'],x['value'])[0],
    'r':stats.pearsonr(x['mu_meas'],x['value'])[0]
}))

dsplot.variable = dsplot.variable.astype('category')
dsplot.model = dsplot.model.astype('category')
dsplot.variable.cat.reorder_categories(['mu_unfit','mu_fit'], inplace=True)
g = sns.FacetGrid(dsplot, col='variable', hue='model', size=4)
g.map(plt.plot, 'mu_meas','value', linestyle='None', marker='o', markeredgecolor='#000000', lw=0.2, markersize=8, alpha=0.75)
for ax in g.axes.flat:
    ax.plot([0,2],[0,2], zorder=1, color='#333333')
    mdl  = ax.get_title().replace('variable = ','')
    mdape= df_perf.loc[mdl].mdape
    sse  = df_perf.loc[mdl].sse
    r    = df_perf.loc[mdl].r
    rho  = df_perf.loc[mdl].rho    
    ax.set_title('%s (MdAPE=%.3g%%, SSE=%.3g)\n(r=%.3g, rho=%.3g)'%(mdl,mdape, sse, r,rho))
g.set_xlabels('Measured growth rate (h^{-1})')
g.set_ylabels('Simulated growth rate (h^{-1})')
g.add_legend()